In [1]:
import os

In [9]:
baudrillard = '/Users/finchmf/coding/TwitterRitual/core/behavior/text/webscraper/baudrillard/textData/simulacra-and-simulation.txt'
baudrillard_2 = '/Users/finchmf/coding/TwitterRitual/core/behavior/text/webscraper/baudrillard/textData/simulations.txt'
def load_data(path: str) -> str:

    text_file = os.path.join(path)
    with open(text_file, "r") as f:
        data = f.read()

    return data

In [10]:
data = load_data(baudrillard)
data_2 = load_data(baudrillard_2)

In [13]:
main_data = data + data_2

In [14]:
main_data

'JEAN BAUDRILLARD \n\n\n\nSimulacra \n\nAND \n\nSimulation \n\n\nTranslated by Sheila Faria Glaser \n\n\nMichigan \n\n\n\n\n\nTABLE OF CONTENTS \n\n\nI. The Precession of Simulacra \n\nII. History: A Retro Scenario \n\nIII. Holocaust \n\nIV. The China Syndrome \n\nV. Apocalypse Now \n\nVI. The Beaubourg Effect: Implosion and Deterrence \n\nVII. Hypermarked and Hypercommodity \n\nVIII. The Implosion of Meaning in the Media \n\nIX. Absolute Advertising, Ground-Zero Advertising \n\nX. Clone Story \n\nXI. Holograms \n\nXII. Crash \n\nXIII. Simulacra and Science Fiction \n\nXIV. The Animals: Territory and Metamorphoses \n\nXV. The Remainder \n\nXVI. The Spiraling Cadaver \n\nXVII. Value\'s Last Tango \n\n\nXVIII. On Nihilism \n\n\nTHE PRECESSION OF SIMULACRA \n\n\nThe simulacrum is never what hides the truth - it is truth that hides the fact that \nthere is none. \n\nThe simulacrum is true. \n\n\n-Ecclesiastes \n\n\nIf once we were able to view the Borges fable in which the cartographers of